In [2]:
import os
import pickle

from modules.logging.format_utils import format_measures
from modules.collecting.results_collector import DataFrameCollector
from modules.logging.logger import DefaultLogger
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode
from modules.algorithms.nn.OSLPP_NN_UTILS import train

In [3]:
with open('experiments/configs/small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [5]:
# sources = ['art', 'clipart', 'product', 'real_world']
# targets = ['art', 'clipart', 'product', 'real_world']

In [6]:
# sources = ['amazon', 'dslr', 'webcam']
# targets = ['amazon', 'dslr', 'webcam']

In [7]:
# sources = ['painting_train', 'real_train', 'sketch_train']
# targets = ['painting_train', 'real_train', 'sketch_train']

In [8]:
# sources = ['train'] 
# targets = ['validation']

In [ ]:
results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
logger = DefaultLogger()
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.15, 0.25]:
                for seed in range(9):
                    params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                    dataset='DomainNet_DCC', source=source, target=target,
                                    num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                    metrics = train(params, lr, epochs, 5, select_reject_mode, seed, common, tgt_private, logger)
                    results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs})
results.get_result().to_csv('../all_results/results/dcc__conf__small__NEW__variable_epochs__multi_mean_std__nn_raw.csv', header=True, index=False)